In [ ]:
#import tensorflow as tf
#import tensorflow.contrib.slim as slim
#eval = slim.evaluation.evaluate_once

from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
                              

In [44]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)


# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(299, 299),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [45]:
from keras.layers import Input
input_img = Input(shape = (299, 299, 3))

In [51]:
from keras.layers import Conv2D, MaxPooling2D, concatenate

conv1_3_3 = Conv2D(32, (3,3), strides=(2,2), activation='relu')(input_img)
conv2_3_3 = Conv2D(32, (3,3), activation='relu')(conv1_3_3)
conv3_3_3 = Conv2D(64, (3,3), padding='same', activation='relu')(conv2_3_3)
pool1_3_3 = MaxPooling2D((3,3), strides=(2,2))(conv3_3_3)
conv3_3_3_reduce = Conv2D(80, (1,1), activation='relu')(pool1_3_3)
conv3_3_3 = Conv2D(192, (3,3), activation='relu')(conv3_3_3_reduce)
pool2_3_3 = MaxPooling2D((3,3), strides=(2,2))(conv3_3_3)

tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(pool2_3_3)
tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(pool2_3_3)
tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(pool2_3_3)
tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

output = concatenate([tower_1, tower_2, tower_3], axis = 3)

#output = pool2_3_3

In [52]:
from keras.layers import Flatten, Dense
#pool3_5_5 = MaxPooling2D((5,5), strides=(3,3))(output)
#conv4_reduce = Conv2D(128, (1,1), activation='relu')(pool3_5_5)
#conv4_3_3 = Conv2D(768, (3,3), activation='relu')(conv4_reduce)
#output = pool3_5_5
output = Flatten()(output)
out    = Dense(2, activation='softmax')(output)

In [53]:
from keras.models import Model
model = Model(inputs = input_img, outputs = out)
print (model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_69 (Conv2D)               (None, 149, 149, 32)  896         input_5[0][0]                    
____________________________________________________________________________________________________
conv2d_70 (Conv2D)               (None, 147, 147, 32)  9248        conv2d_69[0][0]                  
____________________________________________________________________________________________________
conv2d_71 (Conv2D)               (None, 147, 147, 64)  18496       conv2d_70[0][0]                  
___________________________________________________________________________________________

In [54]:
from keras.optimizers import SGD
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate,
          momentum=0.9,
          decay=decay,
          nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

'''

model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=epochs,
          batch_size=32)
'''

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=25,
        validation_data=validation_generator,
        #validation_steps=40 // batch_size)
        validation_steps=20)

Epoch 1/25


KeyboardInterrupt: 

In [ ]:
from keras.models import model_from_json
import os

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights(os.path.join(os.getcwd(), 'model.h5'))

In [ ]:
#scores = model.evaluate(x_test, y_test, verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

scores = model.evaluate_generator(validation_generator, 140)
print("Accuracy: %.2f%%" % (scores[1]*100))